In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
# from config import db_password
import psycopg2
import time
from pathlib import Path
import datetime

In [2]:
# Load the data
file_path = Path('./Resources/Walmart_Store_sales.csv')
Wkly_Sales_df = pd.read_csv(file_path)
Wkly_Sales_df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [3]:
Wkly_Sales_df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [4]:
Wkly_Sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


# ETL Function

In [5]:
    def clean_sales(Wkly_Sales_df):
        # step 1: Removing columns where 90% of values is null   
        try:
            sales_columns_to_keep = [column for column in Wkly_Sales_df.columns if Wkly_Sales_df[column].isnull().sum() < len(Wkly_Sales_df) * 0.9]
            Rev_wkly_sales_df = Wkly_Sales_df[sales_columns_to_keep]
        except Exception as e:
            print('check error-for Null Columns')
            print(e.message)
        try:
        #Step 2: Parsing Date
            #1  drop Null rows
            wkly_sales_dt = Rev_wkly_sales_df['Date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
            # 2 create variables
            date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
            date_form_two = r'\d{4}.[01]\d.[123]\d'
            date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
            date_form_four = r'\d{4}'
            date_form_five = r'\d{4}-{2}\d-{2}\d'
             #3 release date
            Rev_wkly_sales_df['Rev_Date'] = pd.to_datetime(wkly_sales_dt.str.extract(f'({date_form_five}|{date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
        except Exception as e:
            print('check error-for Parse Date converstion')   
            print(e.message)
            return Rev_wkly_sales_df
        # try to concatenate two columns - store and date then remove duplicates
        try:
             Rev_wkly_sales_df['index_id'] = Rev_wkly_sales_df['Store'].astype(str)+'-'+ Rev_wkly_sales_df['Date'].astype(str)
             #Rev_wkly_sales_df.drop_duplicates(subset='index_id', inplace=True)
        except Exception as e:
            print('check error-for duplicate rows')
            print(e.message)
            
            Rev_wkly_sales_df=Rev_wkly_sales_df.dropna()
        
        return Rev_wkly_sales_df

In [6]:
weekly_sales_df=clean_sales(Wkly_Sales_df)
weekly_sales_df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Rev_Date,index_id
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106,2010-01-01,1-05-02-2010
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106,2010-01-01,1-12-02-2010
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106,2010-01-01,1-19-02-2010
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106,2010-01-01,1-26-02-2010
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106,2010-01-01,1-05-03-2010


In [7]:
weekly_sales_df['Month']=pd.DatetimeIndex(weekly_sales_df['Date']).month
weekly_sales_df['Year']=pd.DatetimeIndex(weekly_sales_df['Date']).year
weekly_sales_df['Week']=pd.DatetimeIndex(weekly_sales_df['Date']).weekofyear


In [8]:
weekly_sales_df.head(10)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Rev_Date,index_id,Month,Year,Week
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106,2010-01-01,1-05-02-2010,5,2010,17
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106,2010-01-01,1-12-02-2010,12,2010,48
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106,2010-01-01,1-19-02-2010,2,2010,7
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106,2010-01-01,1-26-02-2010,2,2010,8
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106,2010-01-01,1-05-03-2010,5,2010,18
5,1,12-03-2010,1439541.59,0,57.79,2.667,211.380643,8.106,2010-01-01,1-12-03-2010,12,2010,48
6,1,19-03-2010,1472515.79,0,54.58,2.720,211.215635,8.106,2010-01-01,1-19-03-2010,3,2010,11
7,1,26-03-2010,1404429.92,0,51.45,2.732,211.018042,8.106,2010-01-01,1-26-03-2010,3,2010,12
8,1,02-04-2010,1594968.28,0,62.27,2.719,210.820450,7.808,2010-01-01,1-02-04-2010,2,2010,5
9,1,09-04-2010,1545418.53,0,65.86,2.770,210.622857,7.808,2010-01-01,1-09-04-2010,9,2010,35
